In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
scratchDirectory='/homez/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/CodeFiles/Project_Algorithms/Domain_Profiles/'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data,parcel] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

In [ ]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
if res=='1km':
    outputDirectory=os.path.join(mainDirectory,'Code','OUTPUT','Variable_Calculation','CalculateMoreVariables')
    os.makedirs(outputDirectory, exist_ok=True)
if res=='250m':
    outputDirectory=os.path.join(scratchDirectory,'OUTPUT','Variable_Calculation','CalculateMoreVariables')
    os.makedirs(outputDirectory, exist_ok=True)
    
def initiate_array():
    # Define array dimensions (adjust based on your netCDF)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file=os.path.join(outputDirectory, f'Eulerian_Binary_Array_{res}_{t_res}.h5')
    with h5py.File(out_file, 'a') as f:
        for var_name in ['A_c', 'A_g']:
            if var_name not in f:
                f.create_dataset(var_name, 
                                 (t_size, z_size, y_size, x_size), 
                                 maxshape=(None, z_size, y_size, x_size),
                                 dtype='bool', 
                                 chunks=(1, z_size, y_size, x_size))


            
def add_timestep_at_index(timestep_data_dict, index):
    out_file=os.path.join(outputDirectory, f'Eulerian_Binary_Array_{res}_{t_res}.h5')
    with h5py.File(out_file, 'a') as f:
        if 'A_c' in timestep_data_dict:
            f['A_c'][index] = timestep_data_dict['A_c']
        if 'A_g' in timestep_data_dict:
            f['A_g'][index] = timestep_data_dict['A_g']

In [ ]:
####################################
#RUNNING

In [ ]:
w_thresh1=0.1
w_thresh2=0.5
qcqi_thresh=1e-6

In [ ]:
initiate_array()

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')

    print('calculating variables')
    w_data=data['winterp'].isel(time=t).data
    qc_data=data['qc'].isel(time=t).data
    qi_data=data['qi'].isel(time=t).data
    qcqi_data=qc_data+qi_data
        
    print('calculating A_c binary array')
    cond1 = w_data >= w_thresh2
    cond2 = qcqi_data >= qcqi_thresh
    A_c = np.where(cond1 & cond2, True, False)

    print('calculating A_g binary array')
    cond1 = w_data >= w_thresh1
    cond2 = qcqi_data < qcqi_thresh
    A_g = np.where(cond1 & cond2, True, False)

    print('outputting')
    result = {
        'A_c': A_c,
        'A_g': A_g
    }
    add_timestep_at_index(result, t)


In [ ]:
######################################################

In [ ]:
# #READING BACK IN
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'

# with h5py.File(out_file, 'r') as f:
#     A_c = f['A_c'][t]  # Loads the entire A_c array into memory
#     A_g = f['A_g'][t]  # Loads the entire A_g array into memory

In [ ]:
# #####################
# #TESTING
# w_thresh1=0.1
# w_thresh2=0.5
# qcqi_thresh=1e-6
# z=10
# w_t=data['winterp'].isel(time=t,zh=z).data
# qc_t=data['qc'].isel(time=t,zh=z).data
# qi_t=data['qi'].isel(time=t,zh=z).data
# qcqi_t=qc_t+qi_t

# out_file = dir2 + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'
# with h5py.File(out_file, 'r') as f:
#     A_c = f['A_c'][t,z]  # Loads the entire A_c array into memory
#     A_g = f['A_g'][t,z]  # Loads the entire A_g array into memory

# cond1=(w_t>=w_thresh1)
# cond2=(qcqi_t<qcqi_thresh)
# print(np.all((cond1&cond2)==A_g))

# cond1=(w_t>=w_thresh2)
# cond2=(qcqi_t>=qcqi_thresh)
# print(np.all((cond1&cond2)==A_c))